# Updates

After updating packages, restart the session and then start your notebook from imports.

# Imports

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import pandas as pd

## Reading Json Files

In [2]:
def jsonl_tolist(jsonl_file_path):
    records = []
    with open(jsonl_file_path, 'r') as f:
        for line in f:
            record = json.loads(line)
            records.append(record)
    return records


In [3]:
jsonl_files_path = {
    "train": "train_imdb.jsonl",
    "test": "test_imdb.jsonl",
    "unlabeled": "aug_imdb_unlabeled.jsonl"
}

In [8]:
df_train = pd.DataFrame(jsonl_tolist(jsonl_files_path["train"]))
df_test = pd.DataFrame(jsonl_tolist(jsonl_files_path["test"]))
df_unlabeled = pd.DataFrame(jsonl_tolist(jsonl_files_path["unlabeled"]))

In [11]:
df_train.to_csv("train.csv")
df_test.to_csv("test.csv")
df_unlabeled.to_csv("unlabeled.csv")

In [18]:
df_train["embedding_len"] = df_train["embedding"].apply(len)

In [21]:
df_train["embedding_len"].unique

<bound method Series.unique of 0      768
1      768
2      768
3      768
4      768
      ... 
145    768
146    768
147    768
148    768
149    768
Name: embedding_len, Length: 150, dtype: int64>

In [28]:
def get_feature_columns_name(df):
    features_columns = []
    for i in range(len(df["embedding"][0])):
        name = "feature" + f"{i + 1}"
        features_columns.append(name)
    return features_columns

In [29]:
df_train[get_feature_columns_name(df_train)] = df_train["embedding"].apply(pd.Series)
df_test[get_feature_columns_name(df_test)] = df_test["embedding"].apply(pd.Series)
df_unlabeled[get_feature_columns_name(df_unlabeled)] = df_unlabeled["embedding"].apply(pd.Series)

C:\Users\Mahdi\AppData\Local\Temp\ipykernel_20036\1150993862.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[get_feature_columns_name(df_test)] = df_test["embedding"].apply(pd.Series)
C:\Users\Mahdi\AppData\Local\Temp\ipykernel_20036\1150993862.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[get_feature_columns_name(df_test)] = df_test["embedding"].apply(pd.Series)
C:\Users\Mahdi\AppData\Local\Temp\ipykernel_20036\1150993862.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the re

# Load model

In [ ]:
MODEL_ARGS = {
    'Name': 'microsoft/Phi-3-mini-128k-instruct',
    'DType': bfloat16
}

In [ ]:
def load_model(model_args):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = AutoModelForCausalLM.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
        torch_dtype=torch.model_args['DType'],
        low_cpu_mem_usage=True,
        device_map={"": device},
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
    )

    return model, tokenizer

Good luck :)